<a href="https://colab.research.google.com/github/AmadoMaria/hands-on-supercomputing-with-parallel-computing/blob/master/hackathon_codes_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div align="center"><h1>HPC / SENAI / Hackathon (2022.2)<br>
Brute Force Algorithm </h1></div>

## Introdução

As técnicas de paralelismo compreendem em aplicação de estratégias baseadas na utilização de processamento paralelo manipulando diferentes recursos computacionais. Alguma dessas técnicas compreendem a utilização de bibliotecas paralelas como `OpenMP`, `MPI` e `CUDA`. Cada uma dessas bibliotecas consiste na manipulação de recursos computacionais diferentes, podendo ser utilizadas de forma híbrida, a fim da obtenção de máximo desempenho. No OpenMP e CUDA manipulamos *Threads*, enquanto no MPI, *Processos*, sendo de real relevância os impactos destas unidades de processamento frente aos recursos computacionais alocados. 

A seguir será apresentado um código sequencial para a quebra de senha de até 20 caracteres utilizando um algoritmo de *Força Bruta*. O objetivo básico será inserir técnicas de paralelismo ao código, tal que serão considerados alguns itens nas aplicações finais:

* `Análise dos Custos Computacionais das Aplicações Sequênciais e Paralelas`
    + Profilling CPU (gprof)
    + Profiling GPU (nsys)
* `Estudo das Estruturas Algorítmicas das Aplicações Paralelas`
    + Modelos Algorítmicos Aplicados
    + Características da inserção da API
* `Análise de Desempenho`
     + Experimentação de Parâmetros Ótimos (Melhores valores de Processos, Threads e Grid Computacional)
     + Indices de eficiência (Speedup)

## Regras do Hackathon

* Os idiomas oficiais desse HPC Hackathon são: inglês e português;

* Este ano a competição será em grupos de 2 ou 3 pessoas;

* Tópico Principal: Portabilidade e Otimização de Código;
    
* Os participantes disponibilizarão os resultados através um repositório git pessoal que será configurado pelos participantes e/ou pela ferramenta GOOGLE COLAB;

* Além do código modificado, a resolução deve conter scripts de execução automática para obter os Parâmetros Ótimos e os Speedups;

* O código produzido será avaliado em 2 pontos: desempenho e speedup;

* Os participantes devem codificar e comentar seus códigos;

* Os participantes acessarão o supercomputador via ssh com suas contas previamente configuradas;

* As habilidades necessárias são: Git, Google Colab, Jupyter Notebook, C, C++, Unix, shell, bash, OpenMP, CUDA, MPI;

Boa sorte e boa codificação!

## Código Sequencial 

In [ ]:
%%writefile bruteForce.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <math.h>

//97 to 122 use only lowercase letters
//65 to 90 use only capital letters
//48 to 57 use only numbers

#define START_CHAR 97
#define END_CHAR 122
#define MAXIMUM_PASSWORD 20

long long my_pow(long long x, int y)
{
  long long res = 1;
  if (y==0)
    return res;
  else
    return x * my_pow(x, y-1);
}

void bruteForce(char *pass) 
{
  char force[MAXIMUM_PASSWORD];
  int palavra[MAXIMUM_PASSWORD];
  int pass_b26[MAXIMUM_PASSWORD];
    
  long long int j;
  long long int pass_decimal = 0;
  int base = END_CHAR - START_CHAR + 2;

  int size = strlen(pass);

  for(int i = 0; i < MAXIMUM_PASSWORD; i++)
    force[i] = '\0';

  printf("Try to broke the password: %s\n", pass);

  for(int i = 0; i < size; i++)
    pass_b26[i] = (int) pass[i] - START_CHAR + 1; 

  for(int i = size - 1; i > -1; i--)
    pass_decimal += (long long int) pass_b26[i] * my_pow(base, i);

  long long int max = my_pow(base, size);
  char s[MAXIMUM_PASSWORD];

  for(j = 0; j < max; j++){
    if(j == pass_decimal){
      printf("Found password!\n");
      int index = 0;

      printf("Password in decimal base: %lli\n", j);
      while(j > 0){
        s[index++] = 'a' + j%base-1;
        j /= base;
      }
      s[index] = '\0';
      printf("Found password: %s\n", s);
      break;
    }
  }

}

int main(int argc, char **argv) 
{
  char password[MAXIMUM_PASSWORD];
  strcpy(password, argv[1]);
  time_t t1, t2;
  double dif;

  time (&t1);
    bruteForce(password);
  time (&t2);

  dif = difftime (t2, t1);

  printf("\n%1.2f seconds\n", dif);

  return 0;
}

Writing bruteForce.c


In [ ]:
!chmod 777 bruteForce.c

In [ ]:
!gcc bruteForce.c -o bruteForce -std=c99 -O3

In [ ]:
!./bruteForce senhate

Try to broke the password: senhate
Found password!
Password in decimal base: 2224779850
Found password: senhate

1.00 seconds


## Análise dos Custos Computacionais da Aplicação Sequencial

In [ ]:
!gcc bruteForce.c -o bruteForce -std=c99 -O3 -pg

In [ ]:
!./bruteForce senhate

Try to broke the password: senhate
Found password!
Password in decimal base: 2224779850
Found password: senhate

1.00 seconds


In [ ]:
!gprof -b bruteForce gmon.out

Flat profile:

Each sample counts as 0.01 seconds.
  %   cumulative   self              self     total           
 time   seconds   seconds    calls  Ts/call  Ts/call  name    
100.17      0.87     0.87                             bruteForce


---
## Estudo das Estruturas Algorítmicas das Aplicações Paralelas

### Multicore (OPENMP)

In [ ]:
%%writefile bruteForce.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <math.h>

//97 to 122 use only lowercase letters
//65 to 90 use only capital letters
//48 to 57 use only numbers

#define START_CHAR 97
#define END_CHAR 122
#define MAXIMUM_PASSWORD 20

long long my_pow(long long x, int y)
{
  long long res = 1;
  if (y==0)
    return res;
  else
    return x * my_pow(x, y-1);
}

void bruteForce(char *pass) 
{
  char force[MAXIMUM_PASSWORD];
  int palavra[MAXIMUM_PASSWORD];
  int pass_b26[MAXIMUM_PASSWORD];
    
  long long int j;
  long long int pass_decimal = 0;
  int base = END_CHAR - START_CHAR + 2;

  int size = strlen(pass);

  for(int i = 0; i < MAXIMUM_PASSWORD; i++)
    force[i] = '\0';

  printf("Try to broke the password: %s\n", pass);

  for(int i = 0; i < size; i++)
    pass_b26[i] = (int) pass[i] - START_CHAR + 1; 

  for(int i = size - 1; i > -1; i--)
    pass_decimal += (long long int) pass_b26[i] * my_pow(base, i);

  long long int max = my_pow(base, size);
  char s[MAXIMUM_PASSWORD];

  
  for(j = 0; j < max; j++){
    if(j == pass_decimal){
      printf("Found password!\n");
      int index = 0;

      printf("Password in decimal base: %lli\n", j);
      while(j > 0){
        s[index++] = 'a' + j%base-1;
        j /= base;
      }
      s[index] = '\0';
      printf("Found password: %s\n", s);
      break;
    }
  }

}

int main(int argc, char **argv) 
{
  char password[MAXIMUM_PASSWORD];
  strcpy(password, argv[1]);
  time_t t1, t2;
  double dif;

  time (&t1);
    bruteForce(password);
  time (&t2);

  dif = difftime (t2, t1);

  printf("\n%1.2f seconds\n", dif);

  return 0;
}

Overwriting bruteForce-omp.c


In [ ]:
!chmod 777 bruteForce-omp.c

In [ ]:
!gcc bruteForce-omp.c -o bruteForce-omp -fopenmp -std=c99 -O3

In [ ]:
!OMP_NUM_THREADS=16 ./bruteForce-omp senhate

Try to broke the password: senhate
Found password!
Password in decimal base: 2224779850
Found password: senhate

2.00 seconds


### Multiprocessor (MPI)

In [ ]:
%%writefile buteForce-mpi.c

/**
TODO
*/

In [ ]:
!mpicc bruteForce-mpi.c -o bruteForce-mpi -fopenmp -std=c99 -O3

In [ ]:
!mpirun -np 4 ./bruteForce-mpi senhatez

### Multiprocessor + Multicore (MPI + OpenMP)

In [ ]:
%%writefile buteForce-mpi+omp.c

/**
TODO
*/

In [ ]:
!mpicc bruteForce-mpi+omp.c -o bruteForce-mpi+omp -fopenmp -std=c99 -O3

In [ ]:
!OMP_NUM_THREADS=4 mpirun -np 4 ./bruteForce-mpi+omp senhatez

### GPU (CUDA)

In [ ]:
%%writefile bruteForce-cuda.cu

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <math.h>

//97 to 122 use only lowercase letters
//65 to 90 use only capital letters
//48 to 57 use only numbers

#define START_CHAR 97
#define END_CHAR 122
#define MAXIMUM_PASSWORD 20

__device__ long long my_pow(long long x, int y)
{
  long long res = 1;
  if (y==0)
    return res;
  else
    return x * my_pow(x, y-1);
}

__global__ void bruteForce(char *pass, long long size, long long *result) 
{
  char force[MAXIMUM_PASSWORD];
  int palavra[MAXIMUM_PASSWORD];
  int pass_b26[MAXIMUM_PASSWORD];
    
  long long int j = threadIdx.x;
  long long int pass_decimal = 0;
  int base = END_CHAR - START_CHAR + 2;

  for(int i = 0; i < MAXIMUM_PASSWORD; i++)
    force[i] = '\0';

  printf("Try to broke the password: %s\n", pass);

  for(int i = 0; i < size; i++)
    pass_b26[i] = (int) pass[i] - START_CHAR + 1; 

  for(int i = size - 1; i > -1; i--)
    pass_decimal += (long long int) pass_b26[i] * my_pow(base, i);

  long long int max = my_pow(base, size);

  if(j < max){
    if(j == pass_decimal){
      *result = j;
    }
  }

}

int main(int argc, char **argv) 
{
  char password[MAXIMUM_PASSWORD];
  strcpy(password, argv[1]);
  time_t t1, t2;
  double dif;
  int size = strlen(password);
  long long int *result;

  cudaMallocManaged(&result, sizeof(long long int));

  int NUMBER_OF_BLOCKS = 1;
  int NUMBER_OF_THREADS_PER_BLOCK = size;

  time (&t1);
    bruteForce<<<NUMBER_OF_BLOCKS, NUMBER_OF_THREADS_PER_BLOCK>>>(password, size, result);
    cudaDeviceSynchronize();
  time (&t2);

  
  printf("Found password!\n");
  int index = 0;
  char s[MAXIMUM_PASSWORD];
  int base = END_CHAR - START_CHAR + 2;

  printf("Password in decimal base: %lli\n", *result);
  while(*result > 0){
    s[index++] = 'a' + *result%base-1;
    *result /= base;
  }
  s[index] = '\0';
  printf("Found password: %s\n", s);

  dif = difftime (t2, t1);

  printf("\n%1.2f seconds\n", dif);
  cudaFree(result);

  return 0;
}

Writing bruteForce-cuda.cu


In [ ]:
%%writefile bruteForce-cuda.cu

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>
#include <math.h>

//97 to 122 use only lowercase letters
//65 to 90 use only capital letters
//48 to 57 use only numbers

#define START_CHAR 97
#define END_CHAR 122
#define MAXIMUM_PASSWORD 20

long long my_pow(long long x, int y)
{
  long long res = 1;
  if (y==0)
    return res;
  else
    return x * my_pow(x, y-1);
}

__global__ void bruteForceLoop(long long max, long long pass_decimal, long long *result){
  long long int j = threadIdx.x;
  int base = END_CHAR - START_CHAR + 2;
  char s[MAXIMUM_PASSWORD];
  
  if(j < max){
    if(j == pass_decimal){
      *result = j;
    }
  }
}


void bruteForce(char *pass) 
{
  char force[MAXIMUM_PASSWORD];
  int palavra[MAXIMUM_PASSWORD];
  int pass_b26[MAXIMUM_PASSWORD];
  int size = strlen(pass);  
  
  long long int j = threadIdx.x;
  long long int pass_decimal = 0;
  int base = END_CHAR - START_CHAR + 2;

  for(int i = 0; i < MAXIMUM_PASSWORD; i++)
    force[i] = '\0';

  printf("Try to broke the password: %s\n", pass);

  for(int i = 0; i < size; i++)
    pass_b26[i] = (int) pass[i] - START_CHAR + 1; 

  for(int i = size - 1; i > -1; i--)
    pass_decimal += (long long int) pass_b26[i] * my_pow(base, i);

  long long int max = my_pow(base, size);
  long long int *result;
  cudaMallocManaged(&result, sizeof(long long int));

  int NUMBER_OF_BLOCKS = 1;
  int NUMBER_OF_THREADS_PER_BLOCK = size;

  bruteForceLoop<<<NUMBER_OF_BLOCKS, NUMBER_OF_THREADS_PER_BLOCK>>>(max, pass_decimal, result);
  cudaDeviceSynchronize();
  
  printf("Found password!\n");
  int index = 0;
  char s[MAXIMUM_PASSWORD];


  printf("Password in decimal base: %lli\n", *result);
  while(*result > 0){
    s[index++] = 'a' + *result%base-1;
    *result /= base;
  }
  s[index] = '\0';
  printf("Found password: %s\n", s);

  cudaFree(result);
}


int main(int argc, char **argv) 
{
  char password[MAXIMUM_PASSWORD];
  strcpy(password, argv[1]);
  time_t t1, t2;
  double dif;

  time (&t1);
    bruteForce(password);
  time (&t2);

  dif = difftime (t2, t1);

  printf("\n%1.2f seconds\n", dif);

  return 0;
}

Overwriting bruteForce-cuda.cu


In [ ]:
!chmod 777 bruteForce-cuda.cu

In [ ]:
!nvcc bruteForce-cuda.cu -o bruteForce-cuda -std=c99 -O3

nvcc fatal   : Value 'c99' is not defined for option 'std'


In [ ]:
!nvcc bruteForce-cuda.cu -o bruteForce-cuda -x cu 

bruteForce-cuda.cu(27): warning: variable "base" was declared but never referenced

bruteForce-cuda.cu(28): warning: variable "s" was declared but never referenced

bruteForce-cuda.cu(40): warning: variable "force" was set but never used

bruteForce-cuda.cu(41): warning: variable "palavra" was declared but never referenced

bruteForce-cuda.cu(45): warning: variable "j" was declared but never referenced

/tmp/tmpxft_000000c9_00000000-11_bruteForce-cuda.o: In function `bruteForce(char*)':
tmpxft_000000c9_00000000-6_bruteForce-cuda.cudafe1.cpp:(.text+0x8d): undefined reference to `__device_builtin_variable_threadIdx'
collect2: error: ld returned 1 exit status


In [ ]:
!./bruteForce-cuda senhatez

Found password!
Password in decimal base: 0
Found password: 

0.00 seconds


## Análise de Desempenho

### Parâmetros Ótimos de Execução

1. OpenMP = 72 Threads
2. MPI = 10 nós + 36 Processos
3. MPI + OpenMP = 10 nós + 36 Processos + 4 Threads
4. CUDA = G1D B1DT1D (80 * 32, 1024)

### Tempo de execução em segundos das aplicações

|  Senha (Entradas)    | Sequencial | OpenMP | MPI  | Híbrido | CUDA
| ---------------------| ---------- | ------ | ---  | ------- | ----
| (7z)  zzzzzzz                     |        |        |      |         |  
| (8z)  zzzzzzzz                    |        |        |      |         | 
| (9z)  zzzzzzzzz                   |        |        |      |         | 
| (10z) zzzzzzzzzz                  |        |        |      |         | 

### Speedup

|  Senha    |  OpenMP    | MPI     | Híbrido       | CUDA
| --------- |  ------    | ------  | -------       | ----
| (7z) zzzzzzz        |            |         |               |  
| (8z) zzzzzzzz        |            |         |               |   
| (9z) zzzzzzzzz        |            |         |               |  
| (10z) zzzzzzzzzz       |            |         |               |  

## Conclusões

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.

## Referências Biliográficas

* G. Coulouris, J. Dollimore, T. Kindberg, G.Blair. Distributed Systems: Concepts and Design, Fifth Edition, Pearson, 2011.

* S.Tanenbaum, M. Steen, Distributed Systems: Principles and Paradigms, Second Edition, Pearson, 2006.

* David A. Patterson and John L. Hennessy. Computer Orga- nization and Design: The Hardware/Software Interface. Morgan Kaufmann, 5th Edition, 2013.

* An Introduction to Parallel Programming by Peter S. Pache- co. Morgan Kauffman.

* W. C. Barbosa, An introduction to distributed algorithms, MIT Press, 1997. N. Lynch, Distributed Algorithms, Mit Press, 1996 e Introduction to Distributed Algorithms, Gerard Tel, Cabri- bridge U. Press, 1994.
